# Integrações das Bases de Dados - Equipe GPALT (Analistas de Cardápios)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db


# Dados construídos para integração

O modelo final dos dados propõe a integração de 5 bases de dados diferentes. Essa integração, a partir da estrutura dos dados, não pode ser realizada de maneira tão direta.

O fato de os nomes dos ingredientes nas bases de dados de receitas não serem os nomes dos ingredientes na base do FooDB e não haver nenhuma relação aparente foi um ponto que requereu atenção.

Assim, a solução encontrada foi criar um arquivo de associação entre nomes dos ingredientes em uma base e o nome no FooDB. A construção desse arquivo se deu de maneira totalmente manual, e foi feita a partir dos dados disponíveis.

Importando os dados da associação:

In [2]:
DROP TABLE IF EXISTS Associacao;

CREATE TABLE Associacao (
  IngredienteEmOutraBase VARCHAR(100),
  IngredienteNoFooDB VARCHAR(100),
  PRIMARY KEY (IngredienteEmOutraBase, IngredienteNoFooDB)
) AS SELECT
  nome_ingrediente,nome_foodb
FROM CSVREAD('../data/external/associacaofinal.csv');


Visualizando:

In [3]:
SELECT * FROM Associacao LIMIT 10;


### Número de associações feitas

Um ponto importante é que esse arquivo contém não somente associações aos ingredientes simples do FooDB mas, em alguns casos, associações a ingredientes compostos da base do CulinaryDB. Ele é a chave de todas as associações feitas na base, e é a partir dele que receitas e ingredientes são considerados ou desconsiderados, como será mostrado nos próximos notebooks.

Essa tabela integra 100% dos ingredientes do CulinaryDB, 100% dos ingredientes do Flavor Network e ingredientes com mais de 10000 usos no RecipeNLG.

Além disso, ela foi feita de modo que alterações e integrações novas podem ser feitas para reconstruir a base final. Se esses dados forem incrementados, basta executar todos os notebooks e a base final será reconstruída de forma a refletir esses dados.

Visualizando o número de associações feitas:

In [4]:
SELECT COUNT(*) FROM Associacao;


1669

Há mais de 1600 associações ao todo. Um ponto importante é que essas associações, em alguns casos, apenas repetem o nome do ingrediente do FooDB em letras minúsculas. Para integrar a maior quantidade de dados possível, foi decidido que os nomes de todos os ingredientes seriam convertidos para letras minúsculas primeiro. Assim, qualquer ingrediente presente em uma receita primeiro é pré-processado para ficar em letras minúsculas e, depois disso, em SQL é possível associá-lo.

De todas essas associações, em torno de 800 foram feitas de forma completamente manual.

### Método de elaboração das associações

O ingrediente a ser integrado é `X`. O passo a passo simplificado para integrar `X` é determinar qual das 4 possibilidades se aplica:

1. `X` tem o mesmo nome de algum ingrediente do FooDB, é um sinônimo dele ou é um nome científico dele? Se sim, relacione esse ingrediente.

2. `X` é um subproduto de algum ingrediente do FooDB? Ele é um ingrediente que já existe apenas em uma forma mais específica? Se sim, relacione esse ingrediente.

3. `X` pode ser agrupado em uma classe de alimentos dentro do FooDB? Por exemplo, existe o alimento "Prunus (Cherry, Plum)", que engloba vários alimentos distintos. Se existir uma classe a qual esse ingrediente pertença, relacione esse ingrediente.

4. `X` tem alguma categoria associada na base, como por exemplo no caso de ingredientes do CulinaryDB, que permita associar ele a um ingrediente do FooDB? Se sim, tente relacionar.

5. Se nenhum dos casos aconteceu, provavelmente a associação é difícil. Nesse caso, um dos 4 grupos criados pode ser associado a esse ingrediente: Meat, Plant, Unknown e Essential Oil. Associe esse ingrediente à categoria que ele pertença.

Esses 4 últimos ingredientes garantiram a integração completa. O objetivo deles é evitar que receitas sejam totalmente desconsideradas por terem um ou mais ingredientes não integráveis. Assim, eles servem como base da integração e, portanto, não têm quantidades de nutrientes associadas e nem terão análises feitas a partir deles. Mais detalhes serão mostrados nos próximos notebooks.